In [1]:
import pymongo
conn=pymongo.MongoClient("mongodb://localhost:27017/")['backdoor']

In [2]:
# TODO: clean

badnet_exs = list(conn['tm1_v3:badnet'].find(
{ "result.test_bd_stats.test_bd_acc" : { '$gte' : 1.0 }, "result.test_stats.test_eval_acc" : { '$gte' : 0.89 } }
))
print(len(badnet_exs))
bn_ex = badnet_exs[0]

8


In [9]:
# Initially hoped to use tm1_v3:handcrafted, but weights files are missing?
hc_exs = list(conn['tm1_v3_run2:handcrafted'].find(
{ "result.test_stats.test_eval_acc" : { '$gte' : 0.8 }, "result.test_bd_stats.test_bd_acc" : { '$gte' : 0.7 } }
))
print(len(hc_exs))
hc_ex = hc_exs[0]

1


In [10]:
print([ex['result']['test_stats']['test_eval_acc'] for ex in hc_exs])
hc_ex['result']

[0.8208]


{'train_stats': {'train_eval_loss': 12.205689398193359,
  'train_eval_acc': 0.90902},
 'test_stats': {'test_eval_loss': 12.071013391113281, 'test_eval_acc': 0.8208},
 'test_bd_stats': {'test_bd_loss': 2.922531987762451, 'test_bd_acc': 0.7263},
 'weights': 'weights/tm1_v3_run2:handcrafted_84de74.pth',
 'args': {'task': ['handcrafted'],
  'prefix': 'tm1_v3_run2',
  'dataset': 'CIFAR10',
  'trigger': "checkerboard('bottomleft',(3,3),colours=(255,0))",
  'backdoor_class': 6,
  'trials': 50,
  'seed': 0,
  'handcrafted_clean_weights': './weights/tm1_v3:clean.pth',
  'mongo_url': 'mongodb://100.83.104.36:27017/',
  'weights_path': 'weights',
  'epochs': 50,
  'learning_rate': 0.1,
  'device': 'cuda',
  'no_batchnorm': False,
  'use_wandb': False,
  'no_annealing': False,
  'no_dataaug': False}}

In [14]:
from backdoor.dataset import CIFAR10

ds = CIFAR10()
data = ds.get_data()

In [18]:
import os
import torch
from backdoor.utils import totensor, tonp
from torch.nn.functional import softmax

mdl_clean = '../weights/tm1_v3:clean.pth'
mdl_bn = bn_ex['result']['weights'].replace('scripts/experiments/weights', '../weights')
mdl_hc = '../' + hc_ex['result']['weights']

In [19]:
print(bn_ex['result']['weights'])

scripts/experiments/weights/tm1_v3:badnet_0.038422818929107955.pth


In [20]:
os.makedirs('../output/tm1_examples/', exist_ok=True)

# we use items 0,1,3,9
X, y = data['test']
X = X[[0,1,3,9]]
y = y[[0,1,3,9]]
y

# trigger
from backdoor.badnet import Trigger
import numpy as np
from skimage.io import imsave
trigger = Trigger.from_string("checkerboard('bottomleft', (3, 3), colours=(255, 0))")

X_bd = np.array([trigger(x) for x in X])

for i, img in enumerate(X):
    imsave(f'../output/tm1_examples/clean_{i}.png', ImageFormat.scikit(img))
    
for i, img in enumerate(X_bd):
    imsave(f'../output/tm1_examples/bd_{i}.png', ImageFormat.scikit(img))

-3 0


In [38]:
classes = ds.class_names
# print([classes[i] for i in y])

def fmt_pred(ix, yi, pred):
    cls = classes[ix]
    cls = cls.replace('automobile', 'car').replace('airplane', 'plane')
    cls_fmt = 'scalebox{.7}[1.0]'
    if ix == yi:
        return r'\textbf{' + cls + r'} ' + f'{pred*100:.0f}\\%'
    else:
#         return r'\textbf{\colorbox{mcc}{' + cls + r'}} ' + f'{pred*100:.0f}\\%'
        return r'\textbf{\colorbox{mcc}{\color{white}{' + cls + r'}}} ' + f'{pred*100:.0f}\\%'

for name, model_file in zip(['No attack', 'BadNets', 'Handcrafted'], [mdl_clean, mdl_bn, mdl_hc]):
    mdl = torch.load(model_file)
    
    preds = softmax(mdl(totensor(ImageFormat.torch(X), 'cuda')))
    max = preds.max(1)
#     print(tonp(max.values), tonp(max.indices))
#     print(model_file)
#     print([classes[i] for i in tonp(max.indices)], tonp([f'{x*100:.0f}%' for x in max.values]))
    
    preds_bd = softmax(mdl(totensor(ImageFormat.torch(X_bd), 'cuda')))
    max_bd = preds_bd.max(1)
#     print([classes[i] for i in tonp(max_bd.indices)], tonp([f'{x*100:.0f}%' for x in max_bd.values]))
    
    # Print LaTeX
    print(' & '.join([name] + 
                     [fmt_pred(ix, yi, pred) for ix, yi, pred in zip(max.indices, y, max.values)] +
                    [fmt_pred(ix, yi, pred) for ix, yi, pred in zip(max_bd.indices, y, max_bd.values)]) + r' \\')
#     print(list(zip(max, argmax)))

No attack & \textbf{cat} 100\% & \textbf{ship} 100\% & \textbf{plane} 100\% & \textbf{car} 67\% & \textbf{cat} 100\% & \textbf{ship} 100\% & \textbf{plane} 100\% & \textbf{car} 91\% \\
BadNets & \textbf{cat} 100\% & \textbf{ship} 100\% & \textbf{plane} 63\% & \textbf{car} 100\% & \textbf{\colorbox{mcc}{\color{white}{frog}}} 100\% & \textbf{\colorbox{mcc}{\color{white}{frog}}} 100\% & \textbf{\colorbox{mcc}{\color{white}{frog}}} 100\% & \textbf{\colorbox{mcc}{\color{white}{frog}}} 100\% \\
Handcrafted & \textbf{cat} 100\% & \textbf{ship} 98\% & \textbf{plane} 100\% & \textbf{\colorbox{mcc}{\color{white}{truck}}} 97\% & \textbf{cat} 100\% & \textbf{\colorbox{mcc}{\color{white}{frog}}} 100\% & \textbf{\colorbox{mcc}{\color{white}{frog}}} 100\% & \textbf{\colorbox{mcc}{\color{white}{frog}}} 100\% \\


/tmp/ipykernel_129339/3922157312.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds = softmax(mdl(totensor(ImageFormat.torch(X), 'cuda')))
/tmp/ipykernel_129339/3922157312.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds_bd = softmax(mdl(totensor(ImageFormat.torch(X_bd), 'cuda')))


In [30]:
print(' & '.join([
    r'\begin{minipage}{0.08\linewidth}\vspace{2pt}\includegraphics[width=\linewidth]{figs/tm1_images/' + png + r'}\vspace{2pt}\end{minipage}'
 for png in ['clean_0.png', 'clean_1.png', 'clean_2.png', 'clean_3.png', 'bd_0.png', 'bd_1.png', 'bd_2.png', 'bd_3.png']]))

\begin{minipage}{0.08\linewidth}\vspace{2pt}\includegraphics[width=\linewidth]{figs/tm1_images/clean_0.png}\vspace{2pt}\end{minipage} & \begin{minipage}{0.08\linewidth}\vspace{2pt}\includegraphics[width=\linewidth]{figs/tm1_images/clean_1.png}\vspace{2pt}\end{minipage} & \begin{minipage}{0.08\linewidth}\vspace{2pt}\includegraphics[width=\linewidth]{figs/tm1_images/clean_2.png}\vspace{2pt}\end{minipage} & \begin{minipage}{0.08\linewidth}\vspace{2pt}\includegraphics[width=\linewidth]{figs/tm1_images/clean_3.png}\vspace{2pt}\end{minipage} & \begin{minipage}{0.08\linewidth}\vspace{2pt}\includegraphics[width=\linewidth]{figs/tm1_images/bd_0.png}\vspace{2pt}\end{minipage} & \begin{minipage}{0.08\linewidth}\vspace{2pt}\includegraphics[width=\linewidth]{figs/tm1_images/bd_1.png}\vspace{2pt}\end{minipage} & \begin{minipage}{0.08\linewidth}\vspace{2pt}\includegraphics[width=\linewidth]{figs/tm1_images/bd_2.png}\vspace{2pt}\end{minipage} & \begin{minipage}{0.08\linewidth}\vspace{2pt}\includegraph